# Demo Notebook for using the Reboot Recommendation Engine

Run the cells in order, making sure to enter AWS credentials in the cell when prompted


In [ ]:
#@title Install Python Packages

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@v2.3.1#egg=reboot_toolkit > /dev/null
!echo "Done Installing"

In [ ]:
#@title Import Statements

import reboot_toolkit as rtk
from reboot_toolkit import MovementType, setup_aws, MovementType, MocapType, Handedness, FileType, S3Metadata, PlayerMetadata
import os

In [ ]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
# Also, update the org_id in the field below to your own org_id

boto3_session = setup_aws(org_id="org-mlbbiomech", aws_default_region="us-west-1")

In [ ]:
#@title Set S3 File Info

# Update the below info to match your desired analysis information
mocap_type = MocapType.HAWKEYE_HFR
movement_type = MovementType.BASEBALL_PITCHING
handedness = Handedness.LEFT

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=[mocap_type],
    movement_type=movement_type,
    handedness=handedness,
    file_type=FileType.METRICS_BASEBALL_PITCHING_V2
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Display the Interface for Selecting the Primary Data Segment to Analyze

primary_segment_widget = rtk.create_interactive_widget(s3_df)

display(primary_segment_widget)

In [ ]:
#@title Print the Selections for the Primary Data Segment to Analyze

print('Options selected (re-run the above cell to refresh):')
print(primary_segment_widget.result)

In [ ]:
#@title Set Primary Analysis Segment Info

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'

primary_analysis_segment = PlayerMetadata(
    org_player_ids=primary_segment_widget.result["org_player_ids"],
    session_dates=primary_segment_widget.result["session_dates"],
    session_nums=primary_segment_widget.result["session_nums"],
    session_date_start=primary_segment_widget.result["session_date_start"],
    session_date_end=primary_segment_widget.result["session_date_end"],
    year=primary_segment_widget.result["year"],
    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = rtk.filter_s3_summary_df(primary_analysis_segment, s3_df)

In [ ]:
#@title Add Movement Num and S3 Key to Primary DataFrame to Enable Sorting

prim_available_s3_keys = rtk.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)

primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())

primary_segment_data_df = rtk.merge_data_df_with_s3_keys(primary_segment_data_df, prim_available_s3_keys).sort_values(by=['session_date', 'movement_num'])

In [ ]:
#@title Run metrics against Reboot Recommendation Engine


"""
This is a recommendation engine with the goal of giving coaches guidance on how to help their players improve.
This recommendation engine is trained on player metrics as a predictor of fastball velocity for pitching and bat velocity for hitting. 
Using a random forest like model for training, the recommendation engine then extracts the impact of individual metrics on the overall result.
Given a player's metrics, the engine is then able to recommend aspects to focus on to improve the overall result.
"""
df = rtk.recommendation(boto3_session, primary_segment_data_df, movement_type, mocap_type, handedness)

In [ ]:
df

In [ ]:
fig = rtk.recommendation_violin_plot(boto3_session, primary_segment_data_df, movement_type, mocap_type, handedness, num_features=5)
fig.show()